<a href="https://colab.research.google.com/github/perlihar/INFO284/blob/master/pha033_INFO284_secondassignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import confusion_matrix
from torch.autograd import Variable
from itertools import chain 
import sklearn.metrics as metrics
import matplotlib.pyplot as plt

In [0]:
# check if CUDA is available
use_cuda = torch.cuda.is_available()

In [2]:
BATCH_SIZE = 100
train_set = torchvision.datasets.FashionMNIST("./data", download=True, transform=
                                                transforms.Compose([transforms.ToTensor()]))
test_set = torchvision.datasets.FashionMNIST("./data", download=True, train=False, transform=
                                               transforms.Compose([transforms.ToTensor()]))
trainloader = torch.utils.data.DataLoader(train_set, 
                                           batch_size=BATCH_SIZE)
testloader = torch.utils.data.DataLoader(test_set,
                                          batch_size=BATCH_SIZE)

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw
Processing...
Done!


In [3]:
len(train_set.targets)

60000

In [0]:

class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.l1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.l2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2)
        )
        
        self.fulltconn1 = nn.Linear(64*6*6,600)
        self.fulltconn2 = nn.Linear(600,120)
        self.drop = nn.Dropout(0.25)
        self.fulltconn3 = nn.Linear(120,10)
        
    def forward(self, x):
        out = self.l1(x)
        out = self.l2(out)
        out = out.view(out.size(0), -1)
        out = self.fulltconn1(out)
        out = self.drop(out)
        out = self.fulltconn2(out)
        out = self.fulltconn3(out)
        
        return out


In [5]:


## test the model with 1 batch
model = CNN()
for images, labels in trainloader:
    print("batch size:", images.shape)
    out = model(images)
    print(out.shape)
    break



batch size: torch.Size([100, 1, 28, 28])
torch.Size([100, 10])


In [0]:
learning_rate = 0.001
num_epochs = 5

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## compute accuracy
def get_accuracy(logit, target, batch_size):
    ''' Obtain accuracy for training round '''
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    return accuracy.item()

In [9]:
loss_history = []
for epoch in range(num_epochs):
    train_running_loss = 0.0
    train_acc = 0.0

    model = model.train()

    ## training step
    for i, (images, labels) in enumerate(trainloader):
        
        images = images.to(device)
        labels = labels.to(device)

        ## forward + backprop + loss
        logits = model(images)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()

        ## update model params
        optimizer.step()

        train_running_loss += loss.detach().item()
        train_acc += get_accuracy(logits, labels, BATCH_SIZE)
    
    model.eval()
    print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f' \
          %(epoch, train_running_loss / i, train_acc/i))
    loss_history.append(train_running_loss)

Epoch: 0 | Loss: 0.1848 | Train Accuracy: 93.56
Epoch: 1 | Loss: 0.1704 | Train Accuracy: 94.16
Epoch: 2 | Loss: 0.1558 | Train Accuracy: 94.69
Epoch: 3 | Loss: 0.1495 | Train Accuracy: 94.87
Epoch: 4 | Loss: 0.1352 | Train Accuracy: 95.35


In [14]:
predlist = []
list_of_labels = []
test_acc = 0.0
for i, (images, labels) in enumerate(testloader, 0):
    images = images.to(device)
    
    list_of_labels.append(labels)
    test = Variable(images.view(100, 1, 28, 28))
    outputs = model(test)
    preds = torch.max(outputs, 1)[1].to(device)
    predlist.append(preds)

    labels = labels.to(device)
    outputs = model(images)
    test_acc += get_accuracy(outputs, labels, BATCH_SIZE)
    
print('Test Accuracy: %.2f'%( test_acc/i))

Test Accuracy: 90.88


In [0]:
#print(list_of_labels)
list_of_predictions = [predlist[i].tolist() for i in range(len(predlist))]

new_list_of_labels = [list_of_labels[i].tolist() for i in range(len(list_of_labels))]

list_of_predictions = list(chain.from_iterable(list_of_predictions))
  
new_list_of_labels = list(chain.from_iterable(new_list_of_labels))

In [26]:
matrix = confusion_matrix(new_list_of_labels, list_of_predictions)

print("Classification report for CNN :\n%s\n"
      % (metrics.classification_report(new_list_of_labels, list_of_predictions)))

print('\n\n\n')
print(matrix)
#plt.plot(loss_history)

Classification report for CNN :
              precision    recall  f1-score   support

           0       0.81      0.89      0.85      1000
           1       1.00      0.97      0.98      1000
           2       0.90      0.78      0.83      1000
           3       0.88      0.92      0.90      1000
           4       0.85      0.84      0.84      1000
           5       0.99      0.95      0.97      1000
           6       0.71      0.72      0.72      1000
           7       0.92      0.99      0.95      1000
           8       0.98      0.99      0.98      1000
           9       0.98      0.95      0.96      1000

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000






[[893   0   9  20   1   1  72   1   3   0]
 [  1 973   3  18   2   0   1   0   2   0]
 [ 32   2 776  11  69   0 109   0   1   0]
 [ 27   0   5 921  12   1  32   0   2   0]
 [  4   1  41  39 837   1  75   

A lower learning rate, a higher number of epochs  and a high batch_size increased testing accuracy. Conversely, a higher learning rate caused a severly decreased performance, both on the training score as well as on the test score.

A lower batch_size indicated a greater tendency for overfitting than the reverse. 

---



BATCH_SIZE variation
BATCH_SIZE = 50

lr = 0.001. Checks which is best for that value 
and 10 epochs. Those yielded the best test 
accuracy.

Epoch: 0 | Loss: 0.4028 | Train Accuracy: 86.00

Epoch: 1 | Loss: 0.2813 | Train Accuracy: 89.99

Epoch: 2 | Loss: 0.2465 | Train Accuracy: 91.46

Epoch: 3 | Loss: 0.2223 | Train Accuracy: 92.25

Epoch: 4 | Loss: 0.2027 | Train Accuracy: 92.88

Epoch: 5 | Loss: 0.1844 | Train Accuracy: 93.42

Epoch: 6 | Loss: 0.1697 | Train Accuracy: 94.04

Epoch: 7 | Loss: 0.1528 | Train Accuracy: 94.61

Epoch: 8 | Loss: 0.1446 | Train Accuracy: 94.96

Epoch: 9 | Loss: 0.1321 | Train Accuracy: 95.34

Test Accuracy: 88.41

In [0]:
el

num_epochs variation

RUN 5: lr = 0.01, num_epochs = 10 BATCH_SIZE = 100

Epoch: 0 | Loss: 1.5197 | Train Accuracy: 79.78

Epoch: 1 | Loss: 0.3966 | Train Accuracy: 86.45

Epoch: 2 | Loss: 0.3463 | Train Accuracy: 87.94

Epoch: 3 | Loss: 0.3027 | Train Accuracy: 89.24

Epoch: 4 | Loss: 0.2829 | Train Accuracy: 90.12

Epoch: 5 | Loss: 0.2786 | Train Accuracy: 90.23

Epoch: 6 | Loss: 0.2769 | Train Accuracy: 90.40

Epoch: 7 | Loss: 0.2888 | Train Accuracy: 89.97

Epoch: 8 | Loss: 0.2878 | Train Accuracy: 90.18

Epoch: 9 | Loss: 0.3310 | Train Accuracy: 89.03

Test Accuracy: 87.74

RUN 4:
lr = 0.001 num_epochs = 10 BATCH_SIZE = 100

Epoch: 0 | Loss: 0.4013 | Train Accuracy: 86.15

Epoch: 1 | Loss: 0.2675 | Train Accuracy: 90.59

Epoch: 2 | Loss: 0.2333 | Train Accuracy: 91.76

Epoch: 3 | Loss: 0.2090 | Train Accuracy: 92.78

Epoch: 4 | Loss: 0.1944 | Train Accuracy: 93.22

Epoch: 5 | Loss: 0.1738 | Train Accuracy: 93.95

Epoch: 6 | Loss: 0.1662 | Train Accuracy: 94.23

Epoch: 7 | Loss: 0.1541 | Train Accuracy: 94.68

Epoch: 8 | Loss: 0.1459 | Train Accuracy: 94.97

Epoch: 9 | Loss: 0.1305 | Train Accuracy: 95.52

Test accuracy: 90.30


lr variation:

RUN 3:

lr = 0.1, num_epochs = 5 BATCH_SIZE = 100

Epoch: 0 | Loss: 281.7030 | Train Accuracy: 67.15

Epoch: 1 | Loss: 1.5764 | Train Accuracy: 75.07

Epoch: 2 | Loss: 72807.0001 | Train Accuracy: 66.08

Epoch: 3 | Loss: 335.9824 | Train Accuracy: 69.66

Epoch: 4 | Loss: 190.5280 | Train Accuracy: 72.44

Test Accuracy: 73.97


RUN 2: LR = 0.01, num_epochs = 5 BATCH_SIZE = 100

Epoch: 0 | Loss: 1.8448 | Train Accuracy: 80.34

Epoch: 1 | Loss: 0.3766 | Train Accuracy: 87.06

Epoch: 2 | Loss: 0.3348 | Train Accuracy: 88.48

Epoch: 3 | Loss: 0.2872 | Train Accuracy: 89.91

Epoch: 4 | Loss: 0.2661 | Train Accuracy: 90.67

Test Accuracy: 84.34

Training is overfitting

RUN 1: LR = 0.001 num_epochs = 5 BATCH_SIZE = 100

Epoch: 0 | Loss: 0.3998 | Train Accuracy: 86.49

Epoch: 1 | Loss: 0.2717 | Train Accuracy: 90.48

Epoch: 2 | Loss: 0.2406 | Train Accuracy: 91.62

Epoch: 3 | Loss: 0.2146 | Train Accuracy: 92.56

Epoch: 4 | Loss: 0.1985 | Train Accuracy: 92.98

Test Accuracy: 90.03